In [ ]:
def optimize_cpc_old(self):
    new_cpc = 0
    cv = self.score['cv']
    click = self.score['click']

    if cv == 0:
        if self.score['cost'] > self.target_cpa * 0.7:
            new_cpc = self.target_cpa * 0.6 / click
        else:
            return
    else:
        new_cpc = cv * self.target_cpa / click

    upper = min(self.optimized_cpc * 1.2, 200)
    under = self.optimized_cpc * 0.5
    self.optimized_cpc = min(max(under, new_cpc), upper)

def optimize_cpc_new(self):
    new_cpc = 0
    cv = self.score['cv']
    click = self.score['click']

    if self.score['cost'] > self.target_cpa * 0.7:
        if cv == 0:
            new_cpc = self.target_cpa * 0.6 / click
        else:
            new_cpc = cv * self.target_cpa / click

    upper = min(self.optimized_cpc * 1.2, 200)
    under = self.optimized_cpc * 0.5
    self.optimized_cpc = min(max(under, new_cpc), upper)

def optimize_cpc_new2(self):
    new_cpc = 0
    cv = self.score['cv']
    click = self.score['click']

    if self.score['cost'] > self.target_cpa * 20: # 0.7 => ?
        if cv == 0:
            if self.score['cost'] > self.target_cpa * self.optimize_param['reduction_threthold']:
                new_cpc = self.target_cpa * self.optimize_param['reduction_rate'] / click
            else:
                return
        else:
            new_cpc = cv * self.target_cpa / click

    upper = min(self.optimized_cpc * 1.2, 200)
    under = self.optimized_cpc * 0.5
    self.optimized_cpc = min(max(under, new_cpc), upper)

In [1]:
class Campaign():
    def __init__(self, cpc_logic='old', ctr=0.01, initial_cpc=10, target_cpa=1000, cvr=0.01, 
                 optimize_param={'reduction_threthold': 0.7, 'reduction_rate':0.6},
                 imp=0):
        if cpc_logic == 'old':
            print('Use old')
            self.optimize_cpc_fn = optimize_cpc_old
        else:
            print('Use new logic')
            self.optimize_cpc_fn = optimize_cpc_new
        self.initial_cpc = initial_cpc
        self.target_cpa = target_cpa
        self.ctr = ctr
        self.cvr = cvr
        self.optimize_param = optimize_param
        self.initial_imp = imp
        self.initial_click = ctr * imp
        self.initial_cv = ctr * imp * cvr
        self.reset()
        
    def reset(self):
        self.optimized_cpc = self.initial_cpc
        self.score = {
            'cost': self.initial_click * self.initial_cpc,
            'imp': self.initial_imp,
            'click': self.initial_click,
            'cv': self.initial_cv
            }
        
    def imp(self):
        self.score['imp'] += 1
        if np.random.random() < self.ctr:
            self.score['click'] += 1
            self.score['cv'] += (1 if np.random.rand() < self.cvr else 0)
            self.score['cost'] += self.optimized_cpc
        
    def optimize_cpc(self):
        self.optimize_cpc_fn(self)
        
class CPCSimulator():
    def __init__(self, time, campaigns):
        self.time = time
        self.campaigns = campaigns
        self.results = [{
                'click': np.zeros(time),
                'imp': np.zeros(time),
                'cv': np.zeros(time),
                'cost': np.zeros(time),
                'optimized_cpc': np.zeros(time)
            } for c in campaigns]
        
    def start(self):
        for t in xrange(1, self.time):
            campaign = self.select_campaign(self.campaigns)
            campaign.imp()
            if np.mod(t, 1000) == 0: # 1000imp毎に最適化が走る
                self.optimize()
            self.update_results(t, self.campaigns)
                
    def select_campaign(self, campaigns):
        ecpms = np.array([c.ctr * c.optimized_cpc for c in campaigns])
        weights = ecpms**4 # tetragon
        all_weights = sum(weights)
        v = np.random.random() * all_weights
        
        tmp = 0
        for i, w in enumerate(weights):
            tmp += w
            if v < tmp:
                return campaigns[i]
        print('ERROR')
        return campaigns[-1]
        
    def update_results(self, t, campaigns):
        for i, c in enumerate(campaigns):
            r = self.results[i]
            r['click'][t] = c.score['click']
            r['imp'][t] = c.score['imp']
            r['cv'][t] = c.score['cv']
            r['cost'][t] = c.score['cost']
            r['optimized_cpc'][t] = c.optimized_cpc
    
    def optimize(self):
        for c in self.campaigns:
            c.optimize_cpc()